### 20. JSONデータの読み込み

In [1]:
import gzip
import json

In [2]:
data = []
with gzip.open('data/jawiki-country.json.gz', 'rt') as f:
    for line in f:
        line = line.strip()
        data.append(json.loads(line))

In [3]:
for df in data:
    if df['title'] == 'イギリス':
        text = df['text']
        break

In [4]:
print(text[:1000])

{{redirect|UK}}
{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}
{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク =（[[イギリスの国章|国章]]）
|標語 = {{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Die

### 21. カテゴリ名を含む行を抽出

In [5]:
import re

In [6]:
lines = text.splitlines()

In [7]:
for line in lines:
    if re.search(r'\[\[Category:.*\]\]', line):
        print(line)

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


### 22. カテゴリ名の抽出

In [8]:
for line in lines:
    lst = re.findall(r'\[\[Category:(.*)\]\]', line)
    for category in lst:
        print(category)

イギリス|*
イギリス連邦加盟国
英連邦王国|*
G8加盟国
欧州連合加盟国|元
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


### 23. セクション構造

In [9]:
for line in lines:
    if re.search(r'^==.*==$', line):
        level = len(re.match(r'^=*', line).group()) - 1
        title = re.sub(r'[=\s]', '', line)
        print(level, title)

1 国名
1 歴史
1 地理
2 主要都市
2 気候
1 政治
2 元首
2 法
2 内政
2 地方行政区分
2 外交・軍事
1 経済
2 鉱業
2 農業
2 貿易
2 不動産
2 エネルギー政策
2 通貨
2 企業
3 通信
1 交通
2 道路
2 鉄道
2 海運
2 航空
1 科学技術
1 国民
2 言語
2 宗教
2 婚姻
2 移住
2 教育
2 医療
1 文化
2 食文化
2 文学
2 哲学
2 音楽
3 ポピュラー音楽
2 映画
2 コメディ
2 国花
2 世界遺産
2 祝祭日
2 スポーツ
3 サッカー
3 クリケット
3 競馬
3 モータースポーツ
3 野球
3 カーリング
3 自転車競技
1 脚注
1 関連項目
1 外部リンク


後方参照を使う

In [10]:
for line in lines:
    if x := re.match(r'^(==+)(.*)\1$', line):
        print(len(x[1])-1, x[2].strip())

1 国名
1 歴史
1 地理
2 主要都市
2 気候
1 政治
2 元首
2 法
2 内政
2 地方行政区分
2 外交・軍事
1 経済
2 鉱業
2 農業
2 貿易
2 不動産
2 エネルギー政策
2 通貨
2 企業
3 通信
1 交通
2 道路
2 鉄道
2 海運
2 航空
1 科学技術
1 国民
2 言語
2 宗教
2 婚姻
2 移住
2 教育
2 医療
1 文化
2 食文化
2 文学
2 哲学
2 音楽
3 ポピュラー音楽
2 映画
2 コメディ
2 国花
2 世界遺産
2 祝祭日
2 スポーツ
3 サッカー
3 クリケット
3 競馬
3 モータースポーツ
3 野球
3 カーリング
3 自転車競技
1 脚注
1 関連項目
1 外部リンク


せっかくなのでtreeコマンドっぽいのもやる

In [11]:
def get_sections():
    return [
        (
            len(re.match(r'^=*', line).group()) - 1,
            re.sub(r'[=\s]', '', line)
        )
        for line in lines
        if re.search(r'^==.*==$', line)
    ]

In [12]:
class Section(list):
    def __init__(self, title):
        self.title = title
        super().__init__()
        
    def last(self):
        return self[-1]
    
    def add(self, level, title):
        if level == 1:
            self.append(Section(title))
        else:
            self[-1].add(level-1, title)
            
    def tree_lines(self, head):
        lines = []
        last = len(self) - 1
        for i, x in enumerate(self):
            line = head
            line += '└' if i == last else '├'
            line += x.title
            lines.append(line)
            lines += (x.tree_lines(head + ('　' if i == last else '│')))
        return lines
    
    def __repr__(self):
        return '\n'.join(self.tree_lines(''))

In [13]:
root = Section('root')
for level, title in get_sections():
    root.add(level, title)
root

├国名
├歴史
├地理
│├主要都市
│└気候
├政治
│├元首
│├法
│├内政
│├地方行政区分
│└外交・軍事
├経済
│├鉱業
│├農業
│├貿易
│├不動産
│├エネルギー政策
│├通貨
│└企業
│　└通信
├交通
│├道路
│├鉄道
│├海運
│└航空
├科学技術
├国民
│├言語
│├宗教
│├婚姻
│├移住
│├教育
│└医療
├文化
│├食文化
│├文学
│├哲学
│├音楽
││└ポピュラー音楽
│├映画
│├コメディ
│├国花
│├世界遺産
│├祝祭日
│└スポーツ
│　├サッカー
│　├クリケット
│　├競馬
│　├モータースポーツ
│　├野球
│　├カーリング
│　└自転車競技
├脚注
├関連項目
└外部リンク

### 24. ファイル参照の抽出

In [14]:
for line in lines:
    lst = re.findall(r'\[\[ファイル:([^|\]]*)', line)
    for x in lst:
        print(x)

Royal Coat of Arms of the United Kingdom.svg
United States Navy Band - God Save the Queen.ogg
Descriptio Prime Tabulae Europae.jpg
Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
London.bankofengland.arp.jpg
Battle of Waterloo 1815.PNG
Uk topo en.jpg
BenNevis2005.jpg
Population density UK 2011 census.png
2019 Greenwich Peninsula & Canary Wharf.jpg
Birmingham Skyline from Edgbaston Cricket Ground crop.jpg
Leeds CBD at night.jpg
Glasgow and the Clyde from the air (geograph 4665720).jpg
Palace of Westminster, London - Feb 2007.jpg
Scotland Parliament Holyrood.jpg
Donald Trump and Theresa May (33998675310) (cropped).jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow Terminal 5C Iwelumo-1.jpg
Airbus A380-841 G-XLEB British Airways (10424102995).jpg
UKpop.svg
Anglospeak.svg
Royal Aberdeen Children's Hospital.jpg
CHANDOS3.jpg
The Fabs.JPG
Wembley Stadi

### 25. テンプレートの抽出

In [15]:
for i, line in enumerate(lines):
    if line.startswith('{{基礎情報'):
        start = i
    elif line.startswith('}}'):
        end = i
        break

In [16]:
templete = [
    re.findall(r'\|([^=]*)=(.*)', line)
    for line in lines[start+1 : end]
]
templete = [x[0] for x in templete if x]
dct = {
    key.strip() : value.strip()
    for key, value in templete
}
dct

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />''神よ女王を護り賜え''<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}",
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Linds

### 26. 強調マークアップの除去

In [17]:
dct2 = {
    key : re.sub(r"''+", '', value)
    for key, value in dct.items()
}
dct2

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay H

### 27. 内部リンクの除去

In [18]:
def remove_link(x):
    x = re.sub(r'\[\[[^\|\]]+\|[^{}\|\]]+\|([^\]]+)\]\]', r'\1', x)
    x = re.sub(r'\[\[[^\|\]]+\|([^\]]+)\]\]', r'\1', x)
    x = re.sub(r'\[\[([^\]]+)\]\]', r'\1', x)
    return x

dct3 = {
    key : remove_link(value)
    for key, value in dct2.items()
}
dct3

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:神と我が権利）',
 '国歌': '{{lang|en|God Save the Queen}}{{en icon}}<br />神よ女王を護り賜え<br />{{center|ファイル:United States Navy Band - God Save the Queen.ogg}}',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '英語',
 '首都': 'ロンドン（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首等氏名': 'エリザベス2世',
 '首相等肩書': '首相',
 '首相等氏名': 'ボリス・ジョンソン',
 '他元首等肩書1': '貴族院議長',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等肩書2': '庶民院議長',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}',
 '他元首等肩書3': '最高裁判所長官',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '面積順位': '76',
 '面積大きさ': '1 E11',
 '面積値': '244,820',
 '水面積率': '1.3%',
 '人口統計年': '2018',
 '人口順位': '22',
 '人口大きさ': '1 E7',
 '人口値': '6643万5600<ref>{{Cite web|url=https:/

### 28. MediaWikiマークアップの除去

せっかくなのでその他も取り除いた

In [19]:
def remove_markups(x):
    x = re.sub(r'{{.*\|.*\|([^}]*)}}', r'\1', x)
    x = re.sub(r'<([^>]*)( .*|)>.*</\1>', '', x)
    x = re.sub(r'<[^>]*?/>', '', x)
    x = re.sub(r'\{\{0\}\}', '', x)
    return x

dct4 = {
    key : remove_markups(value)
    for key, value in dct3.items()
}
dct4

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland<ref>英語以外での正式国名:',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': 'Dieu et mon droit（フランス語:神と我が権利）',
 '国歌': 'ファイル:United States Navy Band - God Save the Queen.ogg',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '英語',
 '首都': 'ロンドン（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首等氏名': 'エリザベス2世',
 '首相等肩書': '首相',
 '首相等氏名': 'ボリス・ジョンソン',
 '他元首等肩書1': '貴族院議長',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等肩書2': '庶民院議長',
 '他元首等氏名2': 'Lindsay Hoyle',
 '他元首等肩書3': '最高裁判所長官',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '面積順位': '76',
 '面積大きさ': '1 E11',
 '面積値': '244,820',
 '水面積率': '1.3%',
 '人口統計年': '2018',
 '人口順位': '22',
 '人口大きさ': '1 E7',
 '人口値': '6643万5600',
 '人口密度値': '271',
 'GDP統計年元': '2012',
 'GDP値元': '1兆5478億',
 'GDP統計年MER': '2012',
 'GDP順位MER': '6',
 'GDP値MER': '2兆4337億',
 'GDP統計年': '2012',
 'GDP順位': '6',

### 29. 国旗画像のURLを取得する

In [20]:
import requests

In [21]:
filename = dct4['国旗画像']

session = requests.Session()
url = 'https://en.wikipedia.org/w/api.php'
params = {
    'action' : 'query',
    'format' : 'json',
    'prop' : 'imageinfo',
    'titles' : 'File:' + filename,
    'iiprop' : 'url',
}
r = session.get(url=url, params=params)
data = r.json()
pages = data['query']['pages']
flag_url = pages[list(pages)[0]]['imageinfo'][0]['url']
flag_url

'https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg'

せっかくなので画像も表示させる

In [22]:
from IPython.display import Image
Image(url = flag_url, width = 300)